<a href="https://colab.research.google.com/github/VictorMaRueda/Base-de-datos-modelo-de-quiebra/blob/main/Actividad_6_Proyecto_Final_(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import pandas as pd

# Configurar opciones de visualización
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Cargar las bases de datis
ruta_situacion_financiera = "/content/drive/My Drive/Actividad 6 - Proyecto Final (2)/210030_Estado de situación financiera, corriente_no corriente.xlsx"
situacion_financiera = pd.read_excel(ruta_situacion_financiera)

ruta_resultado_integral = "/content/drive/My Drive/Actividad 6 - Proyecto Final (2)/310030_Estado de resultado integral, resultado del periodo, por funcion de gasto.xlsx"
resultado_integral = pd.read_excel(ruta_resultado_integral)

# Eliminar duplicados en cada base de datos antes de la fusión
situacion_financiera = situacion_financiera.drop_duplicates(subset='Razón social de la sociedad')
resultado_integral = resultado_integral.drop_duplicates(subset='Razón social de la sociedad')

# Fusionar las bases de datos utilizando la columna 'Razón social de la sociedad' como punto en común de ambas listas
datos_combinados = pd.merge(situacion_financiera, resultado_integral, on='Razón social de la sociedad', how='inner')

# Ampliar la capacidad de los resultados a números reales y no enteros
datos_combinados = datos_combinados.fillna(0)
datos_combinados = datos_combinados.replace(0, 1e-6)  # Reemplazar ceros con un valor pequeño cercano a cero

# Calcular KPIs financieros
datos_combinados['Prueba Ácida'] = (datos_combinados['Efectivo y equivalentes al efectivo'] +
                                     datos_combinados['Cuentas comerciales por cobrar y otras cuentas por cobrar corrientes'] +
                                     datos_combinados['Otros activos financieros corrientes']) / datos_combinados['Total pasivos']

datos_combinados['Razón Corriente'] = datos_combinados['Activos corrientes totales'] / datos_combinados['Total pasivos']

datos_combinados['Razón de Liquidez'] = (datos_combinados['Efectivo y equivalentes al efectivo'] +
                                          datos_combinados['Cuentas comerciales por cobrar y otras cuentas por cobrar corrientes'] +
                                          datos_combinados['Otros activos financieros corrientes']) / datos_combinados['Total pasivos']

datos_combinados['Endeudamiento'] = (datos_combinados['Pasivos corrientes totales'] +
                                      datos_combinados['Total de pasivos no corrientes']) / datos_combinados['Total de activos']

datos_combinados['Margen Activo'] = datos_combinados['Ganancia bruta'] / datos_combinados['Ingresos de actividades ordinarias']

datos_combinados['Rentabilidad de Patrimonio'] = datos_combinados['Ganancia (pérdida)'] / datos_combinados['Patrimonio total']

datos_combinados['Rentabilidad de Activo'] = datos_combinados['Ganancia (pérdida)'] / datos_combinados['Total de activos']

# Calcular KPIs de Estado de Resultado Integral
datos_combinados['Margen Bruto'] = datos_combinados['Ganancia bruta'] / datos_combinados['Ingresos de actividades ordinarias']

datos_combinados['Margen Operativo'] = datos_combinados['Ganancia (pérdida) procedente de operaciones continuadas'] / datos_combinados['Ingresos de actividades ordinarias']

datos_combinados['Margen Neto'] = datos_combinados['Ganancia (pérdida)'] / datos_combinados['Ingresos de actividades ordinarias']

datos_combinados['Leverage Total'] = datos_combinados['Total pasivos'] / datos_combinados['Patrimonio total']

# Calcular ratios para Altman's Z-Score
datos_combinados['X1'] = (datos_combinados['Activos corrientes totales'] - datos_combinados['Pasivos corrientes totales']) / datos_combinados['Total de activos']
datos_combinados['X2'] = datos_combinados['Ganancia (pérdida) acumulada anteriormente reconocida en otro resultado integral que surge de la reclasificación de activos financieros de la categoría de medición de valor razonable con cambios en otro resultado integral a la de valor razonable con cambios en resultados'] / datos_combinados['Total de activos']
datos_combinados['X3'] = datos_combinados['Ganancia (pérdida), antes de impuestos'] / datos_combinados['Total de activos']
datos_combinados['X4'] = datos_combinados['Ganancia (pérdida) acumulada anteriormente reconocida en otro resultado integral que surge de la reclasificación de activos financieros de la categoría de medición de valor razonable con cambios en otro resultado integral a la de valor razonable con cambios en resultados'] / datos_combinados['Total pasivos']
datos_combinados['X5'] = datos_combinados['Ingresos de actividades ordinarias'] / datos_combinados['Total de activos']

# Calcular Z-Score de Altman
datos_combinados['Altman Z-Score'] = 1.2 * datos_combinados['X1'] + 1.4 * datos_combinados['X2'] + 3.3 * datos_combinados['X3'] + 0.6 * datos_combinados['X4'] + 1.0 * datos_combinados['X5']

# Definir las categorías
def categorizar_z_score(z_score):
    if z_score < 1.1:
        return "Peligro de quiebra"
    elif 1.2 <= z_score <= 2.59:
        return "Zona de precaución"
    else:
        return "Seguridad ante la quiebra"

# Aplicar la función de categorización a la columna 'Altman Z-Score'
datos_combinados['Categoría Z-Score'] = datos_combinados['Altman Z-Score'].apply(categorizar_z_score)

# Filtrar empresas en peligro de quiebra
peligro_quiebra = datos_combinados[datos_combinados['Categoría Z-Score'] == 'Peligro de quiebra']

# Eliminar duplicados en la lista de empresas en peligro de quiebra
peligro_quiebra = peligro_quiebra.drop_duplicates(subset='Razón social de la sociedad')

# Imprimir la lista completa con categoría
print("Empresas en peligro de quiebra:")
print(peligro_quiebra[['Razón social de la sociedad', 'Altman Z-Score', 'Categoría Z-Score']])

# Contar el número de empresas en peligro de quiebra
num_empresas_en_peligro = peligro_quiebra.shape[0]
print(f"\nNúmero de empresas en peligro de quiebra: {num_empresas_en_peligro}")




Empresas en peligro de quiebra:
                            Razón social de la sociedad  Altman Z-Score  \
0                               OLEODUCTO CENTRAL S.A.S        0.748990   
1                              INVERSIONES MOBEX S.A.S.        0.451458   
2                         Éxito Viajes y Turismo S.A.S.        0.772380   
6                                   CFC GAS HOLDING SAS        0.426571   
8                                OLEODUCTO CENTRAL S.A.        0.802086   
11                               MANUFACTURAS ELIOT SAS        1.083429   
16                                       TEMATEXTIL SAS        0.695357   
18                               PREMIUM PROPERTIES SAS        0.840789   
19                                    AYD PROYECTOS SAS        0.082636   
20                                   CENTURY SPORTS SAS        0.954222   
21                                 ESPACIO Y ESTILO SAS        0.069938   
22                                        DIFERMODA SAS        0.838

**Desarrollo del modelo de predicción de quiebras**

**Paso 1: Empalmamiento de las bases de datos**

En este paso, se descargaron las bases de datos de los Estados de Situación Financiera y los Resultados Integrales del SIIS. Estas bases de datos se cargaron en un notebook de Python, utilizando las bibliotecas de Pandas.

Se buscaba eliminar los datos de razones sociales repetidos, debido a que cada lista cuenta con las mismas filas y con el fin de realizar el analisis solo en el ultimo periodo ( el más reciente ) se excluye la repetición de la razón social para que solo escoja la primera que corresponde al año 2022.

cargando desde: https://siis.ia.supersociedades.gov.co/#/massivereports

los estados financieros y los resultados integrales



Paso 2: Cálculo de KPIs Financieros

En este paso, se calcularon los siguientes KPIs financieros:


Prueba ácida

Prueba ácida = (Activos corrientes - Inventarios) / Pasivos corrientes
La prueba ácida es un indicador de liquidez que mide la capacidad de una empresa para cumplir con sus obligaciones a corto plazo, sin contar con los inventarios.

Razón corriente

Razón corriente = Activos corrientes / Pasivos corrientes
La razón corriente es un indicador de liquidez que mide la capacidad de una empresa para cumplir con sus obligaciones a corto plazo, teniendo en cuenta los inventarios.

Razón de liquidez

Razón de liquidez = (Efectivo + Cuentas por cobrar) / Pasivos corrientes
La razón de liquidez es un indicador de liquidez que mide la capacidad de una empresa para cumplir con sus obligaciones inmediatas, teniendo en cuenta los activos líquidos.

Endeudamiento

Endeudamiento = (Pasivos corrientes + Pasivos no corrientes) / Total activos
El endeudamiento es un indicador de rentabilidad que mide el nivel de endeudamiento de una empresa en relación con sus activos totales.

Margen activo

Margen activo = Ganancia bruta / Ingresos de actividades ordinarias
El margen activo es un indicador de rentabilidad que mide la eficiencia operativa de una empresa en la generación de ingresos.

Rentabilidad de patrimonio

Rentabilidad de patrimonio = Ganancia (pérdida) / Patrimonio total
La rentabilidad de patrimonio es un indicador de rentabilidad que mide la rentabilidad en relación con el patrimonio de una empresa.

Rentabilidad de activo

Rentabilidad de activo = Ganancia (pérdida) / Total de activos
La rentabilidad de activo es un indicador de rentabilidad que mide la eficiencia de los activos en la generación de ganancias.
Paso 3: Cálculo del Z-Score de Altman

En este paso, se calculó el Z-Score de Altman, utilizando las siguientes fórmulas:

X1: (Activos corrientes - Pasivos corrientes) / Total activos
X2: Ganancia acumulada / Total activos
X3: Ganancia antes de impuestos / Total activos
X4: Ganancia acumulada / Total pasivos
X5: Ingresos de actividades ordinarias / Total activos
El Z-Score de Altman es una herramienta financiera que se utiliza para evaluar el riesgo de quiebra de una empresa.

Paso 4: Identificación de Empresas en Peligro de Quiebra

En este paso, se identificaron las empresas en peligro de quiebra, utilizando el siguiente criterio:

Aquellas con puntajes Z inferiores a 1.1 se consideraron en "Peligro de quiebra".
Las empresas identificadas en peligro de quiebra se filtraron y se eliminaron los duplicados para obtener resultados más precisos.

Resultados y Conclusiones

Se generó una lista completa de empresas en peligro de quiebra, que incluye sus puntajes Z y categorías correspondientes. Además, se proporcionó el número total de empresas identificadas en riesgo.

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
!ls "/content/drive/My Drive/Actividad 6 - Proyecto Final (2)"

'210030_Estado de situación financiera, corriente_no corriente.xlsx'
'310030_Estado de resultado integral, resultado del periodo, por funcion de gasto.xlsx'
'Actividad 6 - Proyecto Final (2).ipynb'
 Untitled0.ipynb
